In [2]:
import json
import openai
import datetime
from langame import LangameClient
c = LangameClient(path_to_config_file="../config.yaml")

In [3]:
from langame.conversation_starters import get_existing_conversation_starters
import logging
logger = logging.getLogger("generation")
memes, index, embeddings_model = get_existing_conversation_starters(
    c._firestore_client, logger=logger, confirmed=True)

Skipping id: 4XlXSjPhH6QSGZDzwIQe, garbage
Skipping id: 4sIjdMJr0exliT9yfvuL, garbage
Skipping id: 6jNOw0e7VA6lHWddtqMy, garbage
Skipping id: 9ZfEEkrB3cjLooW6YZM2, garbage
Skipping id: CYsOWtp1ed2XDeplB5O5, garbage
Skipping id: DlXglThZUiPJcuyLLKHV, garbage
Skipping id: EBZCFk0aZPVS1SinJwcz, garbage
Skipping id: FP0UBU9PBzttN2d7YFvZ, garbage
Skipping id: NtXdp7ODn4sR3YR2eEW6, garbage
Skipping id: P05T4keIh8bvF1SKVSSw, garbage
Skipping id: RCI4BcA6Bf1kYBk7hUTe, garbage
Skipping id: U0DUXOSpfqYsgurOyL3L, garbage
Skipping id: Uikgs6eO85MRivL0peL6, garbage
Skipping id: XIPyRwJloK6ZdAdTHOeu, garbage
Skipping id: XdaHW0tJ9wIGAos7WoG4, garbage
Skipping id: bCW1uPMxDEy6iO3oMRQq, garbage
Skipping id: bdf0W2iG9B9rZXbe4xun, garbage
Skipping id: d9FxG5haHnN2ku7spAng, garbage
Skipping id: eyzSmD1AdfKkwOYtfoof, garbage
Skipping id: f1ojlkGYgnOI4MLRGC4T, garbage
Skipping id: fySI8xbjfC5hx7QdHpva, garbage
Skipping id: gLDxkmHFL3OFufeje6E8, garbage
Skipping id: hDybtBa855kDWy9l655v, garbage
Skipping id

Using 8 omp threads (processes), consider increasing --nb_cores if you have more
Launching the whole pipeline 06/26/2022, 13:51:10
There are 2337 embeddings of dim 768
	Compute estimated construction time of the index 06/26/2022, 13:51:10
		-> Train: 16.7 minutes
		-> Add: 0.0 seconds
		Total: 16.7 minutes
	>>> Finished "Compute estimated construction time of the index" in 0.0001 secs
	Checking that your have enough memory available to create the index 06/26/2022, 13:51:10
8.5MB of memory will be needed to build the index (more might be used if you have more)
	>>> Finished "Checking that your have enough memory available to create the index" in 0.0005 secs
	Selecting most promising index types given data characteristics 06/26/2022, 13:51:10
	>>> Finished "Selecting most promising index types given data characteristics" in 0.0000 secs
	Creating the index 06/26/2022, 13:51:10
		-> Instanciate the index HNSW15 06/26/2022, 13:51:10
		>>> Finished "-> Instanciate the index HNSW15" in 0.0023

100%|██████████| 1/1 [00:00<00:00, 242.82it/s]

		>>> Finished "-> Adding the vectors to the index" in 0.0701 secs
	>>> Finished "Creating the index" in 0.0743 secs
	Computing best hyperparameters 06/26/2022, 13:51:10


	>>> Finished "Computing best hyperparameters" in 2.9031 secs
The best hyperparameters are: efSearch=16383
	Compute fast metrics 06/26/2022, 13:51:13
2000
	>>> Finished "Compute fast metrics" in 8.8391 secs
	Saving the index on local disk 06/26/2022, 13:51:22
	>>> Finished "Saving the index on local disk" in 0.0244 secs
Recap:
{'99p_search_speed_ms': 6.235481079995537,
 'avg_search_speed_ms': 4.395972757000089,
 'compression ratio': 0.9576491598101473,
 'index_key': 'HNSW15',
 'index_param': 'efSearch=16383',
 'nb vectors': 2337,
 'reconstruction error %': 0.0,
 'size in bytes': 7496758,
 'vectors dimension': 768}
>>> Finished "Launching the whole pipeline" in 11.9025 secs


In [4]:
out_file_name = f"../data/fine_tune_generation_{datetime.date.today().strftime('%d_%m_%Y')}.jsonl"

for e in memes:
    with open(out_file_name, "a+") as outfile:
        json.dump({
            "prompt": f"{','.join(e['topics'])} ###",
            "completion": f" {e['content']}\n",
        }, outfile)
        outfile.write('\n')

In [5]:
!head -n3 $out_file_name | jq .

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
{
  "prompt": "ice breaker ###",
  "completion": " When is a time when you know for sure you'll soon have to exchange a nice, meaningful conversation with someone new?\n"
}
{
  "prompt": "space exploration,space travel ###",
  "completion": " Do you think humans are the only intelligent life in the universe?\n"
}
{
  "prompt": "ecology ###",
  "completion": " Have natural disasters gotten worse with the increase in human existence? If so, why?\n"
}


In [6]:
!echo "execute: openai tools fine_tunes.prepare_data -f $(pwd)/$out_file_name"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
execute: openai tools fine_tunes.prepare_data -f /Users/louisbeaumont/Documents/langame-worker/notebooks/../data/fine_tune_generation_26_06_2022.jsonl


In [7]:
train_file = openai.File.create(
  file=open("_prepared.jsonl"),
  purpose="fine-tune"
)
ft = openai.FineTune.create(
    training_file=train_file["id"],
    model="curie",
)

In [18]:

get_last_model()

<FineTune fine-tune id=ft-lMkcLUvsQ7SgoRDwYOUESB5J at 0x175648db0> JSON: {
  "created_at": 1656244646,
  "fine_tuned_model": "curie:ft-personal-2022-06-26-12-18-33",
  "hyperparams": {
    "batch_size": 4,
    "learning_rate_multiplier": 0.1,
    "n_epochs": 4,
    "prompt_loss_weight": 0.1
  },
  "id": "ft-lMkcLUvsQ7SgoRDwYOUESB5J",
  "model": "curie",
  "object": "fine-tune",
  "organization_id": "org-KwcHNgfGe4pqdKDLQIJt99UZ",
  "result_files": [
    {
      "bytes": 129038,
      "created_at": 1656245915,
      "filename": "compiled_results.csv",
      "id": "file-OZHUTgo6bBxG1J3gprfEL6p3",
      "object": "file",
      "purpose": "fine-tune-results",
      "status": "processed",
      "status_details": null
    }
  ],
  "status": "succeeded",
  "training_files": [
    {
      "bytes": 323992,
      "created_at": 1656244645,
      "filename": "_prepared.jsonl",
      "id": "file-dc9lP9EWTU2AmsFDXmutv6tK",
      "object": "file",
      "purpose": "fine-tune",
      "status": "proces

In [5]:
results = openai.FineTune.retrieve(
    ft["id"],
)
results

<FineTune fine-tune id=ft-zZ2pHxog8LQ3MqL6W0THUnkn at 0x105903a40> JSON: {
  "created_at": 1649407681,
  "events": [
    {
      "created_at": 1649407681,
      "level": "info",
      "message": "Created fine-tune: ft-zZ2pHxog8LQ3MqL6W0THUnkn",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "learning_rate_multiplier": null,
    "n_epochs": 4,
    "prompt_loss_weight": 0.1
  },
  "id": "ft-zZ2pHxog8LQ3MqL6W0THUnkn",
  "model": "curie",
  "object": "fine-tune",
  "organization_id": "org-KwcHNgfGe4pqdKDLQIJt99UZ",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 204646,
      "created_at": 1649407678,
      "filename": "fine_tune_generation_08_04_2022_train.jsonl",
      "id": "file-R1YxqZwTJObItTy5rKmBPHx1",
      "object": "file",
      "purpose": "fine-tune",
      "status": "uploaded",
      "status_details": null
    }
  ],
  "updated_at": 1649407681,
  "validation_files": 

In [ ]:
!rm -rf _prepared*